In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# import matplotlib.pyplot as plt

In [2]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 1s 0us/step


In [3]:
base_model.trainable = True

In [4]:
def define_model():
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(4, activation='softmax'))
    return model

if __name__ == '__main__':
    model = define_model()
    print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 2,263,108
Trainable params: 2,228,996
Non-trainable params: 34,112
_________________________________________________________________
None


In [5]:
sgd = tf.keras.optimizers.SGD(lr=1e-6, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy' , metrics=['accuracy'])

In [6]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory('data_all/train/', 
                                                 target_size = (224,224), 
                                                 batch_size = 10, 
                                                 class_mode = 'categorical', 
                                                 shuffle = True)

validation_set = test_datagen.flow_from_directory('data_all/validation/', 
                                            target_size = (224,224), 
                                            batch_size = 1, 
                                            class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('data_all/test/', 
                                            target_size = (224,224), 
                                            batch_size = 1, 
                                            class_mode = 'categorical')

Using TensorFlow backend.


Found 17723 images belonging to 4 classes.
Found 2214 images belonging to 4 classes.
Found 2214 images belonging to 4 classes.


In [8]:
checkpoint_path = "training_4/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [9]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 3)

In [10]:
training = model.fit(training_set, 
                     epochs = 30, 
                     validation_data = validation_set, 
                     verbose = 1, 
                     callbacks = [cp_callback, early_stopping])

Epoch 1/30
1773/1773 [==============================] - ETA: 0s - loss: 1.3198 - accuracy: 0.3954
Epoch 00001: saving model to training_4/cp.ckpt
1773/1773 [==============================] - 1921s 1s/step - loss: 1.3198 - accuracy: 0.3954 - val_loss: 0.9806 - val_accuracy: 0.6165
Epoch 2/30
1773/1773 [==============================] - ETA: 0s - loss: 0.9439 - accuracy: 0.6626
Epoch 00002: saving model to training_4/cp.ckpt
1773/1773 [==============================] - 1885s 1s/step - loss: 0.9439 - accuracy: 0.6626 - val_loss: 0.7187 - val_accuracy: 0.8080
Epoch 3/30
1773/1773 [==============================] - ETA: 0s - loss: 0.7392 - accuracy: 0.7913
Epoch 00003: saving model to training_4/cp.ckpt
1773/1773 [==============================] - 1885s 1s/step - loss: 0.7392 - accuracy: 0.7913 - val_loss: 0.5462 - val_accuracy: 0.8812
Epoch 4/30
1773/1773 [==============================] - ETA: 0s - loss: 0.6013 - accuracy: 0.8519
Epoch 00004: saving model to training_4/cp.ckpt
1773/1773 [

1773/1773 [==============================] - ETA: 0s - loss: 0.1239 - accuracy: 0.9687
Epoch 00030: saving model to training_4/cp.ckpt
1773/1773 [==============================] - 1557s 878ms/step - loss: 0.1239 - accuracy: 0.9687 - val_loss: 0.0418 - val_accuracy: 0.9950


In [11]:
results = model.evaluate(test_set)

2214/2214 [==============================] - 182s 82ms/step - loss: 0.0415 - accuracy: 0.9946


In [12]:
model.save('saved_model/MobileNet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/MobileNet/assets


In [13]:
model.save('MobileNet.h5')

In [14]:
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model/MobileNet')
tflite_model = converter.convert()
open("converted_model_mobilenet.tflite", "wb").write(tflite_model)

8947400